In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score
# Cargar el conjunto de datos
data = pd.read_csv('hotel_bookings.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74108 entries, 0 to 74107
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           74108 non-null  object 
 1   is_canceled                     74108 non-null  int64  
 2   lead_time                       74108 non-null  int64  
 3   arrival_date_year               74108 non-null  int64  
 4   arrival_date_month              74108 non-null  object 
 5   arrival_date_week_number        74108 non-null  int64  
 6   arrival_date_day_of_month       74108 non-null  int64  
 7   stays_in_weekend_nights         74108 non-null  int64  
 8   stays_in_week_nights            74108 non-null  int64  
 9   adults                          74108 non-null  int64  
 10  children                        74104 non-null  float64
 11  babies                          74108 non-null  int64  
 12  meal                            

In [12]:
# Imputar valores faltantes
imputer = SimpleImputer(strategy='mean')
data['children'] = imputer.fit_transform(data['children'].values.reshape(-1, 1))

# Codificar variables categóricas
categorical_columns = ['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment',
                       'distribution_channel', 'reserved_room_type', 'assigned_room_type',
                       'deposit_type', 'customer_type', 'reservation_status', 'reservation_status_date']

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

In [14]:
# Definir las características (variables independientes) y la variable objetivo
X = data.drop('is_canceled', axis=1)
y = data['is_canceled']


In [15]:
# División de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [23]:

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')  # Puedes elegir 'mean', 'median', 'most_frequent', o cualquier otra estrategia
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)


# Entrenamiento de modelos
# AdaBoost
adaboost_model = AdaBoostClassifier(n_estimators=50, random_state=42)
adaboost_model.fit(X_train, y_train)

# Gradient Boosting
gradientboost_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gradientboost_model.fit(X_train, y_train)

# XGBoost
xgboost_model = XGBClassifier(n_estimators=100, random_state=42)
xgboost_model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [24]:
# Evaluación de modelos
models = [adaboost_model, gradientboost_model, xgboost_model]
model_names = ['AdaBoost', 'Gradient Boosting', 'XGBoost']

for model, name in zip(models, model_names):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    print(f"Model: {name}")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"ROC AUC: {roc_auc}")
    print("\n")


Model: AdaBoost
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
ROC AUC: 1.0


Model: Gradient Boosting
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
ROC AUC: 1.0


Model: XGBoost
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
ROC AUC: 1.0




tienen un rendimiento perfecto en términos de las métricas de evaluación que proporcionaste (precisión, recall, F1 Score, y ROC AUC). Un rendimiento perfecto, donde todas las métricas son igual a 1.0, es inusual y podría indicar algún problema en proceso de modelado o en la evaluación.

In [30]:
# Realizar validación cruzada y obtener métricas
for model, name in zip(models, model_names):
    # Realizar validación cruzada de 5 pliegues
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

    # Calcular métricas para cada pliegue
    for fold, score in enumerate(scores):
        print(f"Model: {name}, Fold {fold + 1} - Accuracy: {score:.2f}")

    # Calcular métricas en el conjunto de prueba
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    print(f"Model: {name} - Test Set Metrics")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"ROC AUC: {roc_auc:.2f}")
    print("\n")

Model: AdaBoost, Fold 1 - Accuracy: 1.00
Model: AdaBoost, Fold 2 - Accuracy: 1.00
Model: AdaBoost, Fold 3 - Accuracy: 1.00
Model: AdaBoost, Fold 4 - Accuracy: 1.00
Model: AdaBoost, Fold 5 - Accuracy: 1.00
Model: AdaBoost - Test Set Metrics
Accuracy: 1.00
Precision: 1.00
Recall: 1.00
F1 Score: 1.00
ROC AUC: 1.00


Model: Gradient Boosting, Fold 1 - Accuracy: 1.00
Model: Gradient Boosting, Fold 2 - Accuracy: 1.00
Model: Gradient Boosting, Fold 3 - Accuracy: 1.00
Model: Gradient Boosting, Fold 4 - Accuracy: 1.00
Model: Gradient Boosting, Fold 5 - Accuracy: 1.00
Model: Gradient Boosting - Test Set Metrics
Accuracy: 1.00
Precision: 1.00
Recall: 1.00
F1 Score: 1.00
ROC AUC: 1.00


Model: XGBoost, Fold 1 - Accuracy: 1.00
Model: XGBoost, Fold 2 - Accuracy: 1.00
Model: XGBoost, Fold 3 - Accuracy: 1.00
Model: XGBoost, Fold 4 - Accuracy: 1.00
Model: XGBoost, Fold 5 - Accuracy: 1.00
Model: XGBoost - Test Set Metrics
Accuracy: 1.00
Precision: 1.00
Recall: 1.00
F1 Score: 1.00
ROC AUC: 1.00




Los resultados de la validación cruzada y las métricas en el conjunto de prueba muestran que los tres modelos (AdaBoost, Gradient Boosting y XGBoost) tienen un rendimiento perfecto con todas las métricas establecidas en 1.0. Esto sugiere que los modelos están prediciendo con precisión la cancelación de reservas en tus datos de prueba.

Un rendimiento perfecto, aunque es poco común, puede ser válido en situaciones donde los datos son altamente descriptivos y los modelos tienen un ajuste adecuado a los datos.